In [1]:
import os
import argparse
import logging
from pathlib import Path, PurePath
from collections import defaultdict

from src.models.structures import *
from src.models.layers.custom_layers import convnet
from src.models.intermediate_robust_generator.model import *
from src.models.lstm_based.helper import retrieve_dataset, aggregate

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow_addons as tfa

import sklearn as sk
import numpy as np

In [ ]:
BUFFER = 2048
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
EPOCHS = 1

In [ ]:
N_GEN = 3
DATA_PATH = ""
DATASET = 'MNIST'
PARTITIONS = 1000
PARTITION_DIR = "D:\SUMMER_2022\PROJECT\PredictionsFromAggregations\data\interim\lstm"
MODEL_STORE = "D:\SUMMER_2022\PROJECT\PredictionsFromAggregations\models\v1"
SEED = 8008

In [ ]:
name, data = retrieve_dataset(DATASET, None)
x_train, x_test, y_train, y_test = data
dataset = Dataset(name, x_train, x_test, y_train, y_test)

In [ ]:
mean, shape, dataset = aggregate(dataset, PARTITIONS, PARTITION_DIR, SEED)

In [ ]:
if len(shape) == 3:
    a, b, c = shape
    d = 1
else:
    a, b, c, d = shape

n_classes = len(np.unique(dataset.y_train))


if len(dataset.x_train.shape) == 3:
    x_train = np.expand_dims(dataset.x_train, axis=-1)
    x_test = np.expand_dims(dataset.x_test, axis=-1)
else:
    x_train = dataset.x_train
    x_test = dataset.x_test
y_train = tfk.utils.to_categorical(dataset.y_train, n_classes)
y_test = tfk.utils.to_categorical(dataset.y_test, n_classes)

x_train, x_val, y_train, y_val = sk.model_selection.train_test_split(x_train, y_train, test_size=0.1, random_state=42)
"""I present the current worst function in the codebase"""
tf_convert = lambda x, y, types : (tf.data.Dataset.from_tensor_slices((tf.cast(x, types[0]), tf.cast(y, types[1])))).shuffle(BUFFER).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

train_set = tf_convert(x_train, y_train, [tf.float32, tf.uint8])
test_set = tf_convert(x_test, y_test, [tf.float32, tf.uint8])
val_set = tf_convert(x_val, y_val, [tf.float32, tf.uint8])

In [ ]:
intermediate = convnet
config = generator_config((BATCH_SIZE, b, c, d), 10, n_classes, 4, intermediate, None)

model = generator_model(config)

optim = tfk.optimizers.Adam(learning_rate=0.0001)
loss_fn = distance_loss

(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)


In [ ]:
print('Training Model for {} epochs'.format(EPOCHS))

Training Model for 1 epochs


TypeError: Missing required positional argument

In [ ]:
print('Training Model for {} epochs'.format(EPOCHS))

train_acc_store = defaultdict(list)
history = defaultdict(list)

train_acc_metric = tfk.metrics.CategoricalAccuracy()

for epoch in range(EPOCHS):
    print('Epoch {}...'.format(epoch))
    for step, (x_batch, y_batch) in enumerate(train_set): 
        with tf.GradientTape() as tape:
            pred, preds = model(x_batch, training=True)
            weights = [generator.get_layer("generator_dense").weights for generator in model.generators]
            loss_value = loss_fn(y_batch, weights, preds)
        history[epoch].append(loss_value)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optim.apply_gradients(zip(grads, model.trainable_weights))

        train_acc_metric.update_state(y_batch, pred)

    train_acc = train_acc_metric.result()
    train_acc_store[epoch].append(train_acc)
    print("Training acc over epoch: {}, loss: {}".format(float(train_acc), float(loss_value)))

    train_acc_metric.reset_states()

test_acc_metric = tfk.metrics.CategoricalAccuracy()

for x_batch, y_batch in test_set:
    test_pred = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_pred)
test_acc = test_acc_metric.result()

Training Model for 1 epochs
Epoch 0...


ValueError: Exception encountered when calling layer "generator0" (type generator_block).

in user code:

    File "d:\CONDA\envs\tfproba\lib\site-packages\src-0.1.0-py3.9.egg\src\models\layers\custom_layers.py", line 47, in call  *
        if self.intermediate: x = self.intermediate(x)
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "d:\CONDA\envs\tfproba\lib\site-packages\keras\engine\input_spec.py", line 248, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential_10" (type Sequential).
    
    Input 0 of layer "conv2d_40" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (16, 28, 28, 784)
    
    Call arguments received by layer "sequential_10" (type Sequential):
      • inputs=tf.Tensor(shape=(16, 28, 28, 784), dtype=float32